In [6]:
# Importy
import pandas as pd
import json
from sentence_transformers import SentenceTransformer, util

# Wczytujemy dane IFC
ifc_df = pd.read_csv("../data/ifc_sample.csv")

# Wczytujemy klasy CCI
with open("../data/cci_sample.json", "r", encoding="utf-8") as f:
    cci_data = json.load(f)

cci_df = pd.DataFrame(cci_data)

# Ładujemy model embeddingowy
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Generujemy embeddingi
ifc_embeddings = model.encode(ifc_df["Text"].tolist(), convert_to_tensor=True)
cci_embeddings = model.encode(cci_df["full_text"].tolist(), convert_to_tensor=True)

# Dla każdego opisu IFC szukamy najbardziej podobnej klasy CCI
results = []
for i, ifc_emb in enumerate(ifc_embeddings):
    # Liczymy podobieństwa do wszystkich klas
    similarities = util.cos_sim(ifc_emb, cci_embeddings)[0]
    best_idx = similarities.argmax().item()
    best_score = similarities[best_idx].item()
    
    # Dodajemy do wyników
    results.append({
        "GlobalId": ifc_df.loc[i, "GlobalId"],
        "Opis_IFC": ifc_df.loc[i, "Text"],
        "Przewidywany_Kod_CCI": cci_df.loc[best_idx, "code"],
        "Przwidywana_Klasa_CCI": cci_df.loc[best_idx, "name"],
        "Podobieństwo": round(best_score, 4)
    })

# Tworzymy DataFrame z wynikami
results_df = pd.DataFrame(results)
results_df.sort_values("Podobieństwo", ascending=False, inplace=True)
results_df.reset_index(drop=True, inplace=True)

# Podgląd
results_df.head(10)

,GlobalId,Opis_IFC,Przewidywany_Kod_CCI,Przwidywana_Klasa_CCI,Podobieństwo
0,9K0L1M,"Dach dwuspadowy, IfcRoof, Konstrukcja drewnian...",22.11.10,Dachy,0.7450
1,1X2Y3Z,"Ściana zewnętrzna, IfcWall, Ściana zewnętrzna,...",21.12.10,Ściany wewnętrzne,0.7170
2,4A5B6C,"Drzwi jednoskrzydłowe, IfcDoor, Drzwi z drewna...",23.32.10,Drzwi wewnętrzne,0.6835
3,2N3O4P,"Balustrada stalowa, IfcRailing, Balustrada z w...",21.11.10,Ściany zewnętrzne,0.6640
4,8T9U0V,"Drzwi przeciwpożarowe, IfcDoor, Drzwi EI60 sta...",23.32.10,Drzwi wewnętrzne,0.5878
5,0A1B2C,"Strop żelbetowy, IfcSlab, Strop monolityczny z...",21.51.10,Fundamenty,0.5714
6,3E4F5G,"Słup stalowy, IfcColumn, Słup stalowy HEA, wys...",22.31.10,Stropy,0.5318
7,5Q6R7S,"Schody prefabrykowane, IfcStair, Schody żelbet...",22.31.10,Stropy,0.5304
8,7D8E9F,"Okno PCV, IfcWindow, Okno dwuszybowe PCV, rozw...",23.21.10,Okna,0.5208
9,6H7I8J,"Płyta fundamentowa, IfcSlab, Płyta żelbetowa, ...",22.31.10,Stropy,0.4735
